PArt 1: Web Scraping 

In [15]:
import requests
import json
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd

#Store the URL for the wikipedia page
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Get the HTML text for the URL
resp = requests.get(URL)
html_code = resp.text


In [16]:
#Instantiate html parser object
soup = BeautifulSoup(html_code)

#Get the first table on the webpage and all rows within it 
table = soup.find_all('table')[0]
table_rows = table.find_all('tr')

In [38]:
#Add rows values to a list and convert it into a data frame
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df = pd.DataFrame(l, columns=['Postal Code', 'Borough', 'Neighbourhood'])
df = df.iloc[1:]
df = df.replace({'\n':''}, regex=True)

In [58]:
df.shape

(180, 3)

In [59]:
df.head()

,Postal Code,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Part 2: Get Lattitude and Longitude for each postal code

In [61]:
!pip install geocoder

     |████████████████████████████████| 102kB 3.8MB/s 


In [66]:
!pip install pgeocode

In [73]:
import pgeocode
nomi = pgeocode.Nominatim('ca')
lat_log= nomi.query_postal_code("M2A")

In [79]:
latitude = []
longitude = []
for i in range(len(df['Postal Code'])):
  coord= nomi.query_postal_code(df['Postal Code'].iloc[i])
  latitude.append(coord.latitude)
  longitude.append(coord.longitude)

In [81]:
df['latitude'] = latitude
df['longitude'] = longitude

In [82]:
df.head()

,Postal Code,Borough,Neighbourhood,latitude,longitude
1,M1A,Not assigned,Not assigned,NaN,NaN
2,M2A,Not assigned,Not assigned,NaN,NaN
3,M3A,North York,Parkwoods,43.7545,-79.3300
4,M4A,North York,Victoria Village,43.7276,-79.3148
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
